In [1]:
import numpy as np
from scipy.misc import derivative
from scipy.integrate import quad
from typing import Callable, Tuple
import inspect
from R import calc_penalty_matrix

from basis import indicator, polynomial

In [2]:
# import numpy as np
# from scipy.integrate import quad
#
# def indicator(t, l, u):
#     """
#     Computes an indicator function with lower bound l and upper bound u. The output is 1 if the input t is in the range
#     [l, u) or if t equals both l and u, and 0 otherwise.
#
#     Args:
#         t (float): Input value to be evaluated.
#         l (float): Lower bound for the indicator function.
#         u (float): Upper bound for the indicator function.
#
#     Returns:
#         int: 1 if t is in the range [l, u) or if t equals both l and u, 0 otherwise.
#     """
#     return 1 if ((t == u == 1) | ((t >= l) & (t < u))) else 0
#
# def polynomial(t, p=0):
#     """
#     Calculates the value of the polynomial t^p at the given value of t.
#
#     Args:
#         t (float): The input value for the polynomial.
#         p (int): The power to which t is raised. Default is 0.
#
#     Returns:
#         float: The value of the polynomial t^p evaluated at t.
#     """
#     return t ** p
#
# def elements(n, p, l1=0) -> float:
#     """
#     Calculates the integral of a polynomial basis function over a subinterval of [0,1] with length 1/n.
#     The subinterval is determined by the value of l1.
#
#     Args:
#         n (int): The total number of subintervals.
#         p (int): The degree of the polynomial to integrate.
#         l1 (float, optional): The starting point of the subinterval. Defaults to 0.
#
#     Returns:
#         float: The value of the integral.
#     """
#     return quad(lambda t: (indicator(t, l=l1 / n, u=(l1 + 1) / n)) * (polynomial(t, p=p)), l1 / n, (l1 + 1) / n,
#                 full_output=True)[0]
#
# def calc_basis_matrix(x_basis, b_basis) -> np.ndarray:
#     """
#     Calculates the matrix of basis functions for the input dataset.
#
#     Parameters:
#         x_basis (int): The number of subintervals to divide the input data into.
#         b_basis (int): The degree of the polynomial basis functions to use.
#
#     Returns:
#         np.ndarray: A matrix of size (x_basis x b_basis) containing the values of the polynomial basis
#         functions evaluated over the subintervals.
#     """
#     return np.array([[elements(n=x_basis, p=p, l1=l1) for p in range(b_basis)] for l1 in range(x_basis)])

In [3]:
# def elements(f1, f2, low_lim=0, up_lim=1, **kwargs):
#     f1_kwargs = {k:v for k,v in kwargs.items() if k in inspect.signature(f1).parameters}
#     f2_kwargs = {k:v for k,v in kwargs.items() if k in inspect.signature(f2).parameters}
#     return quad(lambda t:(f1(t, **f1_kwargs) * f2(t, **f2_kwargs)), low_lim, up_lim)[0]

In [4]:
# def calc_basis_matrix(basis_for_x, n_x, basis_for_b, n_b):
#     if basis_for_x or basis_for_b is not indicator:
#         return np.array(
#             [[elements(f1=basis_for_x, f2=basis_for_b, low_lim=0, up_lim=1, l=l1, u=(l1+1)/n_x, p=p) for p in range(n_b)] for l1 in range(n_x)])
#     else:
#         return np.array(
#             [[elements(f1=basis_for_x, f2=basis_for_b, low_lim=l1, up_lim=(l1+1)/n_x, l=l1, u=(l1+1)/n_x, p=p) for p in range(n_b)] for l1 in range(n_x)])

In [5]:
# calc_basis_matrix(basis_for_x=indicator, n_x=2, basis_for_b=polynomial, n_b=3)

In [8]:
def indicator(t, l, u):
    return 1 if ((t == u == 1) | ((t >= l) & (t < u))) else 0

def polynomial(t, p=0):
    return t ** p

def basis_function(t, type, l, u, p):
    if type == 'indicator':
        return indicator(t=t, l=l, u=u)
    elif type=='polynomial':
        return polynomial(t=t, p=p)

In [17]:
def elements(n, p, type1, type2, l1=0):
    if type1=='indicator' or type2=='indicator':
        int_lower_lim = l1/n
        int_upper_lim = (l1+1)/n
    else:
        int_lower_lim = 0
        int_upper_lim = 1

    return quad(lambda t: (basis_function(t, type=type1, l=l1/n, u=(l1+1)/n, p=p))*
                          (basis_function(t, type=type2, l=l1/n, u=(l1+1)/n, p=p)),
                int_lower_lim, int_upper_lim,
                full_output=True)[0]

In [18]:
def calc_basis_matrix(x_basis, n_x, b_basis, n_b):
    if x_basis=='indicator' or b_basis=='indicator':
        basis_matrix= np.array(
            [[elements(n=n_x, p=p, type1=x_basis, type2=b_basis, l1=l1) for p in range(n_b)] for l1 in range(n_x)]
        )
    else:
        basis_matrix = np.array(
            [[elements(n=n, p=p, type1=x_basis, type2=b_basis, l1=0) for p in range(n_b)] for n in range(n_x)]
        )
    return basis_matrix

In [19]:
calc_basis_matrix(x_basis='indicator', n_x=2, b_basis='polynomial', n_b=2)

array([[0.5  , 0.125],
       [0.5  , 0.375]])

In [21]:
calc_basis_matrix(x_basis='indicator', n_x=2, b_basis='polynomial', n_b=2)

array([[0.5  , 0.125],
       [0.5  , 0.375]])

---

In [43]:
basis_b=3

R = np.array(
    [[quad(lambda t: ( derivative(polynomial, t, dx=1e-6, n=2, args=[j]) ) *
                     ( derivative(polynomial, t, dx=1e-6, n=2, args=[i]) ),
           0,1,full_output=True)[0] for j in range(basis_b)] for i in range(basis_b)])
zeros_ver = np.zeros(basis_b).reshape(-1,1)
zeros_hor = np.zeros(basis_b+1).reshape(1,-1)
R_0 = np.round(np.concatenate((zeros_hor, np.concatenate((zeros_ver, R), axis=1)), axis=0),3)
R_0

/var/folders/bb/tzggm8cx2f5fyxfhdmmhn7j40000gn/T/ipykernel_19991/1542043282.py:4: DeprecationWarning: scipy.misc.derivative is deprecated in SciPy v1.10.0; and will be completely removed in SciPy v1.12.0. You may consider using findiff: https://github.com/maroba/findiff or numdifftools: https://github.com/pbrod/numdifftools
  [[quad(lambda t: ( derivative(polynomial, t, dx=1e-6, n=2, args=[j]) ) *
/var/folders/bb/tzggm8cx2f5fyxfhdmmhn7j40000gn/T/ipykernel_19991/1542043282.py:5: DeprecationWarning: scipy.misc.derivative is deprecated in SciPy v1.10.0; and will be completely removed in SciPy v1.12.0. You may consider using findiff: https://github.com/maroba/findiff or numdifftools: https://github.com/pbrod/numdifftools
  ( derivative(polynomial, t, dx=1e-6, n=2, args=[i]) ),


array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 4.]])

In [26]:
derivative(polynomial, 0.5, dx=1e-6, n=2, args=[2])

/var/folders/bb/tzggm8cx2f5fyxfhdmmhn7j40000gn/T/ipykernel_19991/457178830.py:1: DeprecationWarning: scipy.misc.derivative is deprecated in SciPy v1.10.0; and will be completely removed in SciPy v1.12.0. You may consider using findiff: https://github.com/maroba/findiff or numdifftools: https://github.com/pbrod/numdifftools
  derivative(polynomial, 0.5, dx=1e-6, n=2, args=[2])


2.0000667788622195

In [45]:
R_0 = calc_penalty_matrix(n_b=5, padding=True)
R_0

array([[ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  4. ,  6. ,  8. ],
       [ 0. ,  0. ,  0. ,  6. , 12. , 18. ],
       [ 0. ,  0. ,  0. ,  8. , 18. , 28.8]])

In [46]:
3*R_0

array([[ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. , 12. , 18. , 24. ],
       [ 0. ,  0. ,  0. , 18. , 36. , 54. ],
       [ 0. ,  0. ,  0. , 24. , 54. , 86.4]])